In [1]:
pip install tensorflow scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

#### Load the data

In [37]:
train_data = pd.read_csv('rdas/train_data.csv')
test_data = pd.read_csv('rdas/test_data.csv')

In [38]:
print(train_data.head())

        time  input_voltage     el_power
0  810.07028           10.0  1228.791720
1  811.06938           10.0  1223.041745
2  812.06848           10.0  1244.960866
3  813.06758           10.0  1229.259058
4  814.06668           10.0  1248.117024


#### Select input(X) and target(y) variables

In [39]:
X_train = train_data[['input_voltage']].values
y_train = train_data['el_power'].values
X_test = test_data[['input_voltage']].values
y_test = test_data['el_power'].values

# Standardize the input features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

### Build a deeper model with more neurons and adjusted dropout

In [40]:
model = Sequential()
model.add(Dense(units=128, activation='relu', input_shape=(X_train_scaled.shape[1],)))
model.add(Dropout(0.3))
model.add(Dense(units=64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=1))  # Output layer (no activation for regression)

# Compile the model with a smaller learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='mean_squared_error',
              metrics=[tf.keras.metrics.RootMeanSquaredError()])


#### Train the model

In [43]:
# from tensorflow.keras.callbacks import EarlyStopping

# early_stop = EarlyStopping(monitor = 'val_loss', patience = 10, restore_best_weights = True)

history = model.fit(
    X_train_scaled, y_train,
    epochs = 100,
    batch_size = 32,
    validation_split = 0.2,
    # callbacks = [early_stop],  # Early stopping callback
    verbose = 2)

Epoch 1/100
1324/1324 - 3s - loss: 181775.7812 - root_mean_squared_error: 426.3517 - val_loss: 18322.8164 - val_root_mean_squared_error: 135.3618 - 3s/epoch - 2ms/step
Epoch 2/100
1324/1324 - 3s - loss: 183481.8750 - root_mean_squared_error: 428.3478 - val_loss: 16427.7227 - val_root_mean_squared_error: 128.1707 - 3s/epoch - 2ms/step
Epoch 3/100
1324/1324 - 3s - loss: 182443.5625 - root_mean_squared_error: 427.1341 - val_loss: 15541.9873 - val_root_mean_squared_error: 124.6675 - 3s/epoch - 2ms/step
Epoch 4/100
1324/1324 - 3s - loss: 182679.9844 - root_mean_squared_error: 427.4108 - val_loss: 16846.2695 - val_root_mean_squared_error: 129.7932 - 3s/epoch - 2ms/step
Epoch 5/100
1324/1324 - 4s - loss: 182635.7812 - root_mean_squared_error: 427.3591 - val_loss: 16300.7568 - val_root_mean_squared_error: 127.6744 - 4s/epoch - 3ms/step
Epoch 6/100
1324/1324 - 3s - loss: 181602.5156 - root_mean_squared_error: 426.1485 - val_loss: 14642.1758 - val_root_mean_squared_error: 121.0049 - 3s/epoch - 3

Epoch 50/100
1324/1324 - 4s - loss: 178783.9219 - root_mean_squared_error: 422.8285 - val_loss: 17344.3145 - val_root_mean_squared_error: 131.6978 - 4s/epoch - 3ms/step
Epoch 51/100
1324/1324 - 3s - loss: 177685.2344 - root_mean_squared_error: 421.5273 - val_loss: 17005.2930 - val_root_mean_squared_error: 130.4043 - 3s/epoch - 2ms/step
Epoch 52/100
1324/1324 - 3s - loss: 176825.8281 - root_mean_squared_error: 420.5066 - val_loss: 18886.0332 - val_root_mean_squared_error: 137.4265 - 3s/epoch - 2ms/step
Epoch 53/100
1324/1324 - 3s - loss: 177020.3906 - root_mean_squared_error: 420.7379 - val_loss: 20114.2070 - val_root_mean_squared_error: 141.8246 - 3s/epoch - 3ms/step
Epoch 54/100
1324/1324 - 3s - loss: 177090.7188 - root_mean_squared_error: 420.8215 - val_loss: 18169.2695 - val_root_mean_squared_error: 134.7934 - 3s/epoch - 3ms/step
Epoch 55/100
1324/1324 - 3s - loss: 177605.4531 - root_mean_squared_error: 421.4326 - val_loss: 16662.6035 - val_root_mean_squared_error: 129.0837 - 3s/epo

Epoch 99/100
1324/1324 - 3s - loss: 174875.7031 - root_mean_squared_error: 418.1814 - val_loss: 20862.1875 - val_root_mean_squared_error: 144.4375 - 3s/epoch - 2ms/step
Epoch 100/100
1324/1324 - 3s - loss: 174576.6875 - root_mean_squared_error: 417.8238 - val_loss: 22255.1992 - val_root_mean_squared_error: 149.1818 - 3s/epoch - 2ms/step


#### Evaluate the model

In [44]:
test_loss, test_rmse = model.evaluate(X_test_scaled, y_test, verbose = 0)

print(f'Test RMSE: {test_rmse:.2f}')

Test RMSE: 426.32


### Make predictions and Visualize Results

In [35]:
y_pred = model.predict(X_test_scaled)

# Compare predicted and actual values
comparison_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred.flatten()})
print(comparison_df.head())

572/572 [==============================] - 1s 1ms/step
        Actual    Predicted
0  1193.140713  1223.898193
1  1107.066421  1223.898193
2  1180.406767  1223.898193
3  1095.551498  1223.898193
4  1177.073719  1223.898193
